In [ ]:
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# 1. SETUP
PROJECT_ID = "altdatapulsedashboard" 
TICKER = "SBUX"                     # Starbucks
BRAND_NAME = "Starbucks"            # As it appears in Consumer Edge

In [ ]:
# 2. GET ALTERNATIVE DATA (Consumer Edge)
print(f"Fetching Spend Data for {BRAND_NAME}...")
sql = f"""
SELECT 
    date,
    spend,
    transactions
FROM `alternative_data.clean_revenue_daily`
WHERE brand = '{BRAND_NAME}'
ORDER BY date
"""
df_alt = pd.read_gbq(sql, project_id=PROJECT_ID)
df_alt['date'] = pd.to_datetime(df_alt['date'])

Fetching Spend Data for Starbucks...


/var/folders/y2/yt_1qz7913jf55m81j1hs4340000gn/T/ipykernel_86423/1928093338.py:12: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_alt = pd.read_gbq(sql, project_id=PROJECT_ID)


In [ ]:
# 3. GET MARKET DATA (Stock Price)
# We use Stock Price as a proxy for "Market Expectation/Performance" for now
print(f"Fetching Market Data for {TICKER}...")
df_market = yf.download(TICKER, start="2020-01-01", end="2025-12-31", progress=False)
df_market = df_market.reset_index()

# Fix for newer yfinance: flatten MultiIndex columns
if isinstance(df_market.columns, pd.MultiIndex):
    df_market.columns = [c[0] for c in df_market.columns]

df_market.columns = [c.lower() for c in df_market.columns]
print(f"Retrieved {len(df_market)} rows of market data")
df_market.head()

Fetching Market Data for SBUX...
Retrieved 1507 rows of market data


,date,close,high,low,open,volume
0,2020-01-02,78.221695,78.221695,77.083610,77.144892,6473800
1,2020-01-03,77.766510,78.002879,77.127426,77.617685,4874100
2,2020-01-06,77.153687,77.398819,76.575891,77.136180,5409800
3,2020-01-07,76.917297,77.066123,76.278214,77.004841,6514700
4,2020-01-08,77.810257,78.221721,76.847260,76.987336,7296300


In [ ]:
# 4. DATA PROCESSING (The "Signal Construction")
# Daily data is noisy. We need a moving average to see the trend.
df_alt['rolling_spend_7d'] = df_alt['spend'].rolling(window=7).mean()
df_alt['rolling_spend_30d'] = df_alt['spend'].rolling(window=30).mean()

In [ ]:
# 5. VISUALIZATION (The "Mosaic" Chart)
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Plot Alt Data (Spend)
fig.add_trace(
    go.Scatter(x=df_alt['date'], y=df_alt['rolling_spend_30d'], name="Consumer Edge Spend (30d Avg)", line=dict(color='blue')),
    secondary_y=False
)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'line': {'color': 'blue'},
              'name': 'Consumer Edge Spend (30d Avg)',
              'type': 'scatter',
              'x': array([], dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': array([], dtype=float64),
              'yaxis': 'y'}],
    'layout': {'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 0.94]},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0]},
               'yaxis2': {'anchor': 'x', 'overlaying': 'y', 'side': 'right'}}
})